In [1]:
import os

import h3
import numpy as np

from mirrorverse.utils import read_data_w_cache, find_neighbors

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [2]:
sql = f'select tag_key, longitude, latitude from mgietzmann_tag_tracks_test'
data = read_data_w_cache(sql)

In [3]:
data['h3_index'] = data.apply(lambda r: h3.geo_to_h3(r['latitude'], r['longitude'], 4), axis=1)
data.head()

,tag_key,longitude,latitude,h3_index
0,159002b,-151.15,59.675,840c51dffffffff
1,159002b,-152.25,59.625,840c503ffffffff
2,159002b,-152.45,59.625,840c503ffffffff
3,159002b,-152.35,59.625,840c503ffffffff
4,159002b,-152.30,59.625,840c503ffffffff


In [4]:
data['neighbors'] = data['h3_index'].apply(lambda i: len(find_neighbors(100, i)))
data.head()

,tag_key,longitude,latitude,h3_index,neighbors
0,159002b,-151.15,59.675,840c51dffffffff,19
1,159002b,-152.25,59.625,840c503ffffffff,19
2,159002b,-152.45,59.625,840c503ffffffff,19
3,159002b,-152.35,59.625,840c503ffffffff,19
4,159002b,-152.30,59.625,840c503ffffffff,19


In [5]:
data['ll'] = np.log(1 / data['neighbors'])
data.head()

,tag_key,longitude,latitude,h3_index,neighbors,ll
0,159002b,-151.15,59.675,840c51dffffffff,19,-2.944439
1,159002b,-152.25,59.625,840c503ffffffff,19,-2.944439
2,159002b,-152.45,59.625,840c503ffffffff,19,-2.944439
3,159002b,-152.35,59.625,840c503ffffffff,19,-2.944439
4,159002b,-152.30,59.625,840c503ffffffff,19,-2.944439


In [6]:
np.exp(data.groupby('tag_key')['ll'].mean().reset_index()['ll'].mean())

0.05203887455732783